In [1]:
import time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from skimage.morphology import label
from scipy.ndimage import measurements
import itertools

from z3 import *

In [2]:
url='https://www.janestreet.com/puzzles/pent-up-frustration-index/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
y =[text for text in soup.body.stripped_strings]
#display([[i,j] for i,j in enumerate(y)])
display(Markdown("### "+y[7]+"\n\n"+str("\n".join(y[10:18]))))

# First steps to do the pentomino filling and work out the score.
#
# How to move to actually getting the optimum score is a much harder question
# https://www.jstage.jst.go.jp/article/ipsjjip/23/3/23_252/_pdf/-char/ja


### 'Pent-up' Frustration

Place as many distinct pentominoes as you want into an 8-by-8 grid, in such a
way that the placement is “tight” — i.e., no piece(s) can freely slide around
within the grid.
The score for a given placement is the
sum of the square roots of the areas of
the empty regions
in the grid.
What is the largest score you can obtain?
This month, when you send in your entry, please send in your grid. Please use
the
standard
notation
, — i.e. F,
I, L, N, P, T, U, V, W, X, Y, Z — and use “.” to denote empty spaces. (So the
top row in the valid placement example would be “…Z..LL”)

<img src='https://www.janestreet.com/puzzles/20181101_pent-up.png' width =300>

In [3]:
def reset(positions):
    min_x, min_y = min(positions, key=lambda x:x[::-1])
    return tuple(sorted((x-min_x, y-min_y) for x, y in positions))

def variation(positions):
    return list({reset(var) for var in (
        positions,
        [(-y,  x) for x, y in positions], # Anti-clockwise 90
        [(-x, -y) for x, y in positions], # 180
        [( y, -x) for x, y in positions], # Clockwise 90

        [(-x,  y) for x, y in positions], # Mirror vertical
        [(-y, -x) for x, y in positions], # Mirror diagonal
        [( x, -y) for x, y in positions], # Mirror horizontal
    )})

def orientation(x,y,shape,A,B):
    for (alpha,beta) in shape:
        links = []
        for (i,j) in shape:
            if (i,j) !=(alpha,beta):
                if  (x+i-alpha >= 0) & (x+i-alpha <A) & (y+j-beta >=0) &  (y+j-beta <B):
                     links.append((x+i-alpha,y+j-beta))
    
            if len(links) == len(shape)-1:
                yield links
            
def forced(x,y,A,B,n):
    shapes = [
    (((0, 1), (1, 0), (1, 1), (1, 2), (2, 0)), "F"),
    (((0, 0), (0, 1), (0, 2), (0, 3), (0, 4)), "I"),
    (((0, 0), (0, 1), (0, 2), (0, 3), (1, 3)), "L"),
    (((0, 2), (0, 3), (1, 0), (1, 1), (1, 2)), "N"),
    (((0, 0), (0, 1), (0, 2), (1, 0), (1, 1)), "P"),
    (((0, 0), (1, 0), (1, 1), (1, 2), (2, 0)), "T"),
    (((0, 0), (0, 1), (1, 1), (2, 0), (2, 1)), "U"),
    (((0, 0), (0, 1), (0, 2), (1, 2), (2, 2)), "V"),
    (((0, 0), (0, 1), (1, 1), (1, 2), (2, 2)), "W"),
    (((0, 1), (1, 0), (1, 1), (1, 2), (2, 1)), "X"),
    (((0, 1), (1, 0), (1, 1), (1, 2), (1, 3)), "Y"),
    (((0, 0), (1, 0), (1, 1), (1, 2), (2, 2)), "Z")
    ]

    shape_variations = {shape: variation(shape) for shape, name in [shapes[n-1]]}

    for key,vals in shape_variations.items():
        poss = []
        for val in vals:
            poss += [*orientation(x,y,val,A,B)]
    
        return poss

def place(x,y,n,X,N):
    return Or([
        And([X[i,j]==n for i,j in force]) 
        for force in forced(x,y,N[0],N[1],n)
            ])

def neighbours(i,j,N):
    l=[]
    if i-1 >= 0:
        l.append((i-1,j))
    if i+1 < N[0]:
        l.append((i+1,j))
    if j-1 >= 0:
        l.append((i,j-1))
    if j+1 < N[1]:
        l.append((i,j+1))
    return l

In [4]:
# some setup and tests

N= [8,8]
blank = [(3, 3), (4, 3), (3, 4), (4, 4)]
blank = [(0,0),(0,2),(0,4),(0,5),(0,7),
 (1,4),(1,5),
 (2,0),(2,2),(2,7),
 (3,3),(3,5),(3,7),
 (4,0),(4,1),
 (5,3),(5,5),(5,7),
 (6,0),(6,2),
 (7,0),(7,4),(7,6),(7,7)]
len(blank)

24

In [13]:
start = time.time()
#s = Tactic('default').solver()
s = Optimize()
X = np.array([[Int("X_%s%s" % (i+1,j+1)) for j in range(N[1]) ] for i in range(N[0]) ],dtype=object)

# force blank/non-blank
#s += [X[i,j] == 0 if (i,j) in blank else X[i,j] !=0 for j in range(N[1]) for i in range(N[0]) ]

# force number of blank cells and number of solo blank cells
s += PbEq([(X[i,j] == 0,1) for j in range(N[1]) for i in range(N[0])],24)
s += Sum([If(X[i,j]==0, 
             If(Product([X[k,l] for (k,l) in neighbours(i,j,N)])==0,1,0),0)
          for j in range(N[1]) for i in range(N[0]) ]) < 13

# the big one. Set the placements round a given cell for each pentomino
s += [Implies(X[i,j] == n,place(i,j,n,X,N)) for n in range(1,13) for j in range(N[1]) for i in range(N[0])]

#limit the cells to 0 (blank) or 1-12 (pentomino)
s += [Or([X[i,j]==n for n in range(0,13)]) for j in range(N[1]) for i in range(N[0]) ]

#either 5 or 0 of each number
s += [Or(PbEq([(X[i,j]==n,1) for j in range(N[1]) for i in range(N[0])],5),
         PbEq([(X[i,j]==n,1) for j in range(N[1]) for i in range(N[0])],0))
         for n in range(1,13)]

# starting coding non-slip.
s += [Sum([X[i,j] for i in range(N[0])]) > 0 for j in range(N[1])]
s += [Sum([X[i,j] for j in range(N[1])]) > 0 for i in range(N[0])]

print("SETUP DONE .. in {:0.4f} seconds".format(time.time()-start))

SETUP DONE .. in 4.3883 seconds


In [ ]:
start= time.time()
max = 0
max_grid = []
while time.time()-start < 5:
    if s.check() == sat:
        m = s.model()
        x = np.array([[m.evaluate(X[i,j]).as_long() for j in range(N[1])] for i in range(N[0])])
        labels, num = measurements.label(x==0)
        areas = measurements.sum(x==0, labels, index=range(1, num+1))
        if np.sum(areas**.5) > max:
            max = np.sum(areas**.5)
            print(max)
            max_grid = x
    
    s += Or([X[i,j]!=int(x[i,j]) for j in range(N[1]) for i in range(N[0])])
    
fig,ax = plt.subplots(1,1,figsize=(4,4)) 
y = np.array(max_grid).astype('int').astype('str')
shapes = [' ','F', 'I', 'L', 'N', 'P', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
mapping = np.vectorize(lambda x:shapes[x])
shading = mapping(max_grid)
sns.heatmap(max_grid,annot=shading,cbar=False,cmap="terrain_r",fmt="",linewidths=1,center=3,linecolor="grey",annot_kws={"size":12})
plt.show()
ax.axis("off")

print('Solution took {:0.4f} seconds'.format(time.time()-start))
labels, num = measurements.label(max_grid==0,output=int)
areas = measurements.sum(max_grid==0, labels, index=range(1, num+1))
print("The areas are:{} giving {:.3f} as a solution".format(areas,np.sum(areas**.5)))

In [ ]:
url='https://www.janestreet.com/puzzles/pent-up-frustration-solution/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
y =[text for text in soup.body.stripped_strings]
#display([[i,j] for i,j in enumerate(y)])
display(Markdown("### "+y[8]+"\n\n"+str("\n".join(y[10:14]))))

<img src="https://www.janestreet.com/puzzles/20181203_pent-up_ans.png" width = 200>